In [1]:
import numpy as np
import pandas as pd

import math
import random
import operator
import itertools

from deap import algorithms,base,creator,tools,gp
from features1 import fill_age_1,add_title

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_x = train.drop("Survived",axis=1)
train_y = train['Survived']



In [2]:
def MungeData(data):
    # Sex
#    data=add_title(data)   
#    data=fill_age_1(data)
#    data.drop('Title', axis=1, inplace=True)
    data.drop(['Ticket', 'Name', 'PassengerId'], inplace=True, axis=1)
    data.Sex.fillna('0', inplace=True)
    data.loc[data.Sex != 'male', 'Sex'] = 0
    data.loc[data.Sex == 'male', 'Sex'] = 1
    # Cabin
    data.Cabin.fillna('0', inplace=True)
    data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 1
    data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 2
    data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 3
    data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 4
    data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 5
    data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 6
    data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 7
    data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 8
    # Embarked
    data.loc[data.Embarked == 'C', 'Embarked'] = 1
    data.loc[data.Embarked == 'Q', 'Embarked'] = 2
    data.loc[data.Embarked == 'S', 'Embarked'] = 3
    data.Embarked.fillna(0, inplace=True)
    data.fillna(-1, inplace=True)

    return data.astype(float)
train_x = MungeData(train_x).values.tolist()
test = MungeData(test).values.tolist()
train_y = train_y.tolist()

In [3]:
def proba(data):
    return (1.-(1./(1.+np.exp(-data))))

def predict(prob):
    return np.round(prob)

def protectedDiv(a,b):
    try:
        return a/b
    except ZeroDivisionError:
        return 1
def protectedLog(a):
    x=np.log(a)
    if np.isnan(x):
        return 728
    return x

pset = gp.PrimitiveSetTyped('MAIN',itertools.repeat(float, 8), float)
pset.addPrimitive(protectedLog, [float], float)
pset.addPrimitive(operator.add, [float,float],float)
pset.addPrimitive(operator.sub, [float,float],float)
pset.addPrimitive(operator.mul, [float,float],float)
pset.addPrimitive(protectedDiv, [float,float],float)
pset.addPrimitive(operator.neg, [float],float)
pset.addPrimitive(np.cos,[float],float)
pset.addPrimitive(np.sin,[float],float)
pset.addPrimitive(np.tanh,[float],float)
pset.addPrimitive(np.minimum, [float,float],float)
pset.addPrimitive(np.maximum, [float,float],float)
# Terminals
pset.addEphemeralConstant("rand1", lambda: random.random()*2-1, float)
#pset.addEphemeralConstant("rand1000", lambda: round(random.random()*1000),float)
pset.addTerminal(np.pi,float)
pset.addTerminal(2.0,float)
pset.renameArguments(ARG0='PClass')
pset.renameArguments(ARG1='Sex')
pset.renameArguments(ARG2='Age')
pset.renameArguments(ARG3='SibSp')
pset.renameArguments(ARG4='Parch')
pset.renameArguments(ARG5='Fare')
pset.renameArguments(ARG6='Cabin')
pset.renameArguments(ARG7='Embarked')

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset = pset, min_=1, max_=5)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

In [5]:
def log_loss(y_true, y_prob):
    n = len(y_true)
    result=0.0
    for i in range(n):
        y_prob[i]=np.minimum(np.maximum(1e-15, y_prob[i]),1-1e-15)
        if y_true[i]:
            result+=np.log(y_prob[i])
        else:
            result+=np.log(1-y_prob[i])
    return 999.0 if np.isnan(result) else -1.0*result/n

def evalFitness(individual):
    func = toolbox.compile(expr=individual)
    return log_loss(train_y,[proba(func(*x)) for x in train_x]),
    
toolbox.register('evaluate', evalFitness)

In [20]:
def staticLimitCrossover(ind1, ind2, heightLimit):
    keepInd1, keepInd2 = toolbox.clone(ind1), toolbox.clone(ind2)
    gp.cxOnePoint(ind1, ind2)
    if ind1.height > heightLimit:
        ind1[:] = keepInd1
    if ind2.height > heightLimit:
        ind2[:] = keepInd2
        
def staticLimitCrossover1(ind1, ind2, heightLimit):
    keepInd1, keepInd2 = toolbox.clone(ind1), toolbox.clone(ind2)
    gp.cxOnePointLeafBiased(ind1, ind2, 0.2)
    if ind1.height > heightLimit:
        ind1[:] = keepInd1
    if ind2.height > heightLimit:
        ind2[:] = keepInd2

def staticLimitMutation(individual, expr, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutUniform(individual, expr,pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation1(individual, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutEphemeral(individual, mode='one')
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation2(individual, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutNodeReplacement(individual, pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation3(individual, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutInsert(individual, pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation4(individual, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutShrink(individual)
    if individual.height > heightLimit:
        individual[:] = keepInd 
toolbox.register('selectW', tools.selWorst)
toolbox.register('selectR', tools.selRoulette)
toolbox.register('selectT', tools.selTournament, tournsize=3)
toolbox.register('selectB', tools.selBest)
toolbox.register('selectSPEA', tools.selSPEA2)
toolbox.register('select', tools.selDoubleTournament)


toolbox.register("mate", staticLimitCrossover, heightLimit=17)
toolbox.register("mateL", staticLimitCrossover1, heightLimit=17)
toolbox.register("mutateS",  staticLimitMutation4, heightLimit=17) 
toolbox.register("mutateI",  staticLimitMutation3, pset=pset, heightLimit=17) 
toolbox.register("mutateR",  staticLimitMutation2, pset=pset, heightLimit=17) 
toolbox.register("mutateE",  staticLimitMutation1, heightLimit=17) 
toolbox.register("mutateU", staticLimitMutation, expr=toolbox.expr_mut, pset=pset, heightLimit=17) 

def print_hof(hof):
    result=[str(hof[i].fitness.values) for i in range(5)]
    print " ".join(result)
def same_hof(h1,h2):
    for i in range(len(h1)):
        if not h1[i][0]==h2[i][0]:
            return False
    return True    
    

In [24]:
size=500
pop = toolbox.population(n=size)
hof = tools.HallOfFame(5)
MUT_R = 0.2
MUT_I = 0.2
MUT_U = 0.2
MUT_E = 0.2
MUT_S = 0.2
NGEN = 40000

fitnesses  = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
current_best = toolbox.selectB(pop,5)

for i in range(5):
    hof.insert(current_best[i])

count=10
for g in range(NGEN):
    # clone population
    offspring = map(toolbox.clone, pop)

    # Mate
    random.shuffle(offspring)
    a=toolbox.selectR(offspring,len(offspring)/2)
    for c1,c2 in zip(a[::2],a[1::2]):
         if not c1==c2:
            toolbox.mate(c1,c2)
            del c1.fitness.values
            del c2.fitness.values

    # Mutate
    #b = toolbox.selectW(offspring, size/2)
    for mutant in offspring:
        if random.random()<0.1:
            if random.random()<MUT_I:
                toolbox.mutateI(mutant)
            if random.random()< MUT_R:
                toolbox.mutateR(mutant)
            if random.random()< MUT_U:
                toolbox.mutateU(mutant)
            if random.random()< MUT_E:
                toolbox.mutateE(mutant)
            if random.random()< MUT_S:
                toolbox.mutateS(mutant)

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        #print fit
        ind.fitness.values = fit
        
    pop[:]=offspring
    hof_t=map(toolbox.clone, hof)
    hof.update(pop)
    # Print fitness score per generation
    if count ==10:
        result=[str(hof[i].fitness.values)[1:-2] for i in range(5)]
        result = " ".join(result)
        result = str(g) + " " + result
        print result
        count=0
    count+=1
#    print str(gp.PrimitiveTree(hof[0]))
#    if same_hof(hof_t,hof):
#        count+=1
#    else:
#        count =0
#    if count>30:
        

0 0.57113992936882529 0.57141996751466084 0.57292908389662822 0.57932736864518519 0.57932736864518519
10 0.54241132232603628 0.54241132232603628 0.56776353236331079 0.56947273669341669 0.56947273669341669
20 0.54241132232603628 0.54241132232603628 0.56776353236331079 0.56776353236331079 0.56947273669341669
30 0.54241132232603628 0.54241132232603628 0.56776353236331079 0.56776353236331079 0.56776353236331079
40 0.54241132232603628 0.54241132232603628 0.55784275229443703 0.55784275229443703 0.56776353236331079
50 0.52650523409509864 0.54241132232603628 0.54241132232603628 0.55784275229443703 0.55784275229443703
60 0.52650523409509864 0.54241132232603628 0.54241132232603628 0.55086996829457846 0.55784275229443703
70 0.52650523409509864 0.54241132232603628 0.54241132232603628 0.55086996829457846 0.55086996829457846
80 0.52650523409509864 0.54241132232603628 0.54241132232603628 0.55086996829457846 0.55086996829457846
90 0.52650523409509864 0.54241132232603628 0.54241132232603628 0.550869968

790 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
800 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
810 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
820 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
830 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
840 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
850 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
860 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
870 0.51448702037159244 0.52650523409509864 0.52689431769660233 0.52689431769660233 0.52689431769660233
880 0.51448702037159244 0.52650523409509864 0.52689431769660233 

1580 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1590 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1600 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1610 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1620 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1630 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1640 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1650 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1660 0.51448702037159244 0.51719881542212021 0.51719881542212021 0.51719881542212021 0.51719881542212021
1670 0.51448702037159244 0.51719881542212021 0.51719881

2370 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2380 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2390 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2400 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2410 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2420 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2430 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2440 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2450 0.51448702037159244 0.51668544841097286 0.51719881542212021 0.51719881542212021 0.51719881542212021
2460 0.51448702037159244 0.51668544841097286 0.51719881

3160 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3170 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3180 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3190 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3200 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3210 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3220 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3230 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3240 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.50862322117473002 0.51448702037159244
3250 0.50862322117473002 0.50862322117473002 0.50862322

...

17280 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17290 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17300 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17310 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17320 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17330 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17340 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17350 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17360 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
17370 0.47414496780469612 0.47414496780469612 

18060 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18070 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18080 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18090 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18100 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18110 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18120 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18130 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18140 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18150 0.47414496780469612 0.47414496780469612 

18840 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18850 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18860 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18870 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18880 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18890 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18900 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18910 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18920 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
18930 0.47414496780469612 0.47414496780469612 

19620 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19630 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19640 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19650 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19660 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19670 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19680 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19690 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19700 0.47414496780469612 0.47414496780469612 0.47414496780469612 0.49055916397549426 0.49726196039718407
19710 0.47414496780469612 0.47414496780469612 

KeyboardInterrupt: 

In [ ]:
    for mutant in offspring:
        if random.random()<0.01:
            r = random.random()
            if r<MUT_I:
                toolbox.mutateI(mutant)
            elif r< MUT_I+MUT_R:
                toolbox.mutateR(mutant)
            elif r< MUT_I+MUT_R+MUT_U:
                toolbox.mutateU(mutant)
            else:
                toolbox.mutateE(mutant)
                

In [25]:
str(gp.PrimitiveTree(hof[0]))


'add(add(sub(Sex, 3.141592653589793), PClass), sub(mul(Age, minimum(SibSp, 3.141592653589793)), sub(protectedDiv(maximum(3.141592653589793, Embarked), Age), tanh(3.141592653589793))))'